# 데이터 import

In [197]:
import pandas as pd
import numpy as np

In [198]:
train = pd.read_csv("data/train_df.csv", encoding='cp949')
test = pd.read_csv("data/test_df.csv", encoding='cp949')

In [199]:
train.head()

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT
0,0,1129000014045300,5011000220046300,패션의류,상의,3
1,1,1135000009051200,5011000178037300,생활/건강,반려동물,3
2,2,1135000030093100,5011000265091400,패션의류,기타패션의류,9
3,3,1154500002014200,5011000315087400,식품,농산물,10
4,4,1165000021008300,5011000177051200,식품,가공식품,3


In [200]:
test.head()

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM
0,32000,5013000043028400,1165000021097200,식품,농산물
1,32001,5013000044016100,1154500002066400,식품,농산물
2,32002,5013000205030200,4139000102013200,식품,농산물
3,32003,5013000205030200,4221000040093400,식품,농산물
4,32004,5013000268011400,2726000004017100,식품,농산물


In [201]:
train.isnull().sum()

index            0
SEND_SPG_INNB    0
REC_SPG_INNB     0
DL_GD_LCLS_NM    0
DL_GD_MCLS_NM    0
INVC_CONT        0
dtype: int64

### train_df.csv
+ index : 인덱스
+ SEND_SPG_INNB : 송하인_격자공간고유번호
+ REC_SPG_INNB : 수하인 격자공간고유번호
+ DL_GD_LCLS_NM : 카테고리_대
+ DL_GD_MCLS_NM : 카테고리_중
+ INVC_CONT : 운송장 건 수 => Label

### test_df.csv
+ index : 인덱스
+ SEND_SPG_INNB : 송하인_격자공간고유번호
+ REC_SPG_INNB : 수하인 격자공간고유번호
+ DL_GD_LCLS_NM : 카테고리_대
+ DL_GD_MCLS_NM : 카테고리_중

### sample_submission.csv
+ index : 인덱스
+ INVC_CONT : 운송장 건 수

# 데이터 전처리

In [202]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset

In [203]:
# 원핫 인코딩()
train_one = pd.get_dummies(train)
test_one = pd.get_dummies(test)

In [204]:
train_x = train_one.drop('INVC_CONT',axis = 1).values.astype("float32")
train_y = train_one['INVC_CONT'].values.astype("float32")

test_x = test_one.values.astype("float32")

In [205]:
from sklearn.model_selection import train_test_split
train_x, valid_x, train_y, valid_y = train_test_split(train_x, 
                                                      train_y,
                                                      # stratify = train_y,
                                                      # stratify는 분류 문제에서 split되는 비율을 결정함
                                                      # 이건 유통량 예측이기 때문에 고려 X?
                                                      random_state = 17, 
                                                      test_size = 0.15)

In [206]:
print("Train x's Shape : {}".format(train_x.shape))
print("Train y's Shape : {}".format(train_y.shape))
print("Valid x's Shape : {}".format(valid_x.shape))
print("Valid x's Shape : {}".format(valid_y.shape))

Train x's Shape : (27200, 29)
Train y's Shape : (27200,)
Valid x's Shape : (4800, 29)
Valid x's Shape : (4800,)


In [207]:
train_x = torch.tensor(train_x)
train_y = torch.tensor(train_y)
valid_x = torch.tensor(valid_x)
valid_y = torch.tensor(valid_y)

In [171]:
train_x_tensor = torch.tensor(train_x)
train_y_tensor = torch.tensor(train_y)
train_tensor = TensorDataset(train_x_tensor, train_y_tensor)
train_loader = DataLoader(train_tensor, batch_size = 128, num_workers = 0, shuffle = True)

valid_x_tensor = torch.tensor(valid_x)
valid_y_tensor = torch.tensor(valid_y)
valid_tensor = TensorDataset(valid_x_tensor, valid_y_tensor)
valid_loader = DataLoader(valid_tensor, batch_size = 128, num_workers = 0, shuffle = True)

test_x_tensor = torch.tensor(test_x)

In [208]:
print("Train x's Shape : {}".format(train_x_tensor.shape))
print("Train y's Shape : {}".format(train_y_tensor.shape))
print("Valid x's Shape : {}".format(valid_x_tensor.shape))
print("Valid x's Shape : {}".format(valid_y_tensor.shape))

Train x's Shape : torch.Size([27200, 29])
Train y's Shape : torch.Size([27200])
Valid x's Shape : torch.Size([4800, 29])
Valid x's Shape : torch.Size([4800])


# 모델 설계

In [194]:
def train(model, train_loader, opt):
    model.train()
    for batch_idx, (data, label) in enumerate(train_loader):
        data, label = data.cuda(), label.cuda()
        opt.zero_grad()
        output = model(data).squeeze()       
        loss = torch.sqrt(nn.MSELoss()(output, label))
        loss.backward()
        opt.step()
        
        if batch_idx % 100 == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(epoch, batch_idx * len(data), len(train_loader.dataset), 100. * batch_idx / len(train_loader), loss.item()))

def evaluate(model, loader):
    model.eval()
    
    valid_loss = 0
    corrent = 0
    
    with torch.no_grad():
        for data, label in loader:
            data, label = data.cuda(), label.cuda()
            output = model(data)
            valid_loss += nn.MSELoss()(output, label).sum().item()
        
        valid_loss /= len(loader.dataset)
        return valid_loss

## 모델 1
그냥 단순한 다층 선형 신경망을 설계

In [241]:
class MyModel1(nn.Module):
    def __init__(self):
        super(MyModel1, self).__init__()
        self.fc1 = nn.Linear(29, 16)
        self.btn1 = nn.BatchNorm1d(16)
        self.fc2 = nn.Linear(16, 8)
        self.btn2 = nn.BatchNorm1d(8)
        self.fc3 = nn.Linear(8 ,1)
        
        self.act_fn = nn.ELU()
    def forward(self, x):
        x = self.fc1(x)
        x = self.btn1(x)
        x = self.act_fn(x)
        
        x = self.fc2(x)
        x = self.btn2(x)
        x = self.act_fn(x)
        
        x = self.fc3(x)
        return x
    

model = MyModel1().cuda()
print(model)

MyModel1(
  (fc1): Linear(in_features=29, out_features=16, bias=True)
  (btn1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (btn2): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=8, out_features=1, bias=True)
  (act_fn): ELU(alpha=1.0)
)


In [242]:
opt = optim.Adam(model.parameters())
Loss_fn = nn.MSELoss()
EPOCHS = 2000

# for epoch in range(1, EPOCHS+1):
#     train(model, train_loader, opt)
#     valid_loss = evaluate(model, valid_loader)
#     print("[EPOCH: {}], \tValidation Loss: {:.4f}\n".format(epoch, valid_loss))
    
for epoch in range(1, EPOCHS+1):
    model.train()
    x = train_x; label = train_y
    x = x.cuda(); label = label.cuda()
    
    opt.zero_grad()
    output = model(x).squeeze()
#     print(output[2].item(),label[2].item())
    loss = Loss_fn(output, label)
    loss.backward()
    opt.step()
    
     
    model.eval()
    output = model(valid_x.cuda())
    valid_loss = Loss_fn(output, valid_y.cuda())
    
    print('Epoch: {}\t Loss: {:.6f}\t valid_Loss : {:.6f}\n'.format(epoch, loss, valid_loss))

C:\anaconda3\envs\JSY_GPU\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([4800])) that is different to the input size (torch.Size([4800, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 1	 Loss: 54.260704	 valid_Loss : 39.696232

Epoch: 2	 Loss: 54.171173	 valid_Loss : 41.067238

Epoch: 3	 Loss: 54.094189	 valid_Loss : 41.530380

Epoch: 4	 Loss: 54.025219	 valid_Loss : 41.710770

Epoch: 5	 Loss: 53.954823	 valid_Loss : 41.749977

Epoch: 6	 Loss: 53.882553	 valid_Loss : 41.735451

Epoch: 7	 Loss: 53.807037	 valid_Loss : 41.701008

Epoch: 8	 Loss: 53.729237	 valid_Loss : 41.657021

Epoch: 9	 Loss: 53.650112	 valid_Loss : 41.604805

Epoch: 10	 Loss: 53.570480	 valid_Loss : 41.544735

Epoch: 11	 Loss: 53.490513	 valid_Loss : 41.480865

Epoch: 12	 Loss: 53.410183	 valid_Loss : 41.427525

Epoch: 13	 Loss: 53.332508	 valid_Loss : 41.383656

Epoch: 14	 Loss: 53.262543	 valid_Loss : 41.348099

Epoch: 15	 Loss: 53.198235	 valid_Loss : 41.320755

Epoch: 16	 Loss: 53.137890	 valid_Loss : 41.301067

Epoch: 17	 Loss: 53.080666	 valid_Loss : 41.284363

Epoch: 18	 Loss: 53.026073	 valid_Loss : 41.256741

Epoch: 19	 Loss: 52.973324	 valid_Loss : 41.215122

Epoch: 20	 Loss: 52.9


Epoch: 188	 Loss: 44.736633	 valid_Loss : 29.981667

Epoch: 189	 Loss: 44.687996	 valid_Loss : 30.372978

Epoch: 190	 Loss: 44.638527	 valid_Loss : 30.354673

Epoch: 191	 Loss: 44.589199	 valid_Loss : 30.258394

Epoch: 192	 Loss: 44.540680	 valid_Loss : 30.540247

Epoch: 193	 Loss: 44.492470	 valid_Loss : 30.015873

Epoch: 194	 Loss: 44.443531	 valid_Loss : 30.318045

Epoch: 195	 Loss: 44.394356	 valid_Loss : 29.905125

Epoch: 196	 Loss: 44.344456	 valid_Loss : 29.930992

Epoch: 197	 Loss: 44.295261	 valid_Loss : 30.190969

Epoch: 198	 Loss: 44.246716	 valid_Loss : 29.979790

Epoch: 199	 Loss: 44.197830	 valid_Loss : 30.384819

Epoch: 200	 Loss: 44.148888	 valid_Loss : 30.142925

Epoch: 201	 Loss: 44.099880	 valid_Loss : 30.317884

Epoch: 202	 Loss: 44.051201	 valid_Loss : 30.034779

Epoch: 203	 Loss: 44.002773	 valid_Loss : 30.064508

Epoch: 204	 Loss: 43.954678	 valid_Loss : 29.741470

Epoch: 205	 Loss: 43.906773	 valid_Loss : 29.812967

Epoch: 206	 Loss: 43.859016	 valid_Loss : 29.

Epoch: 372	 Loss: 37.336254	 valid_Loss : 24.566717

Epoch: 373	 Loss: 37.309181	 valid_Loss : 24.428900

Epoch: 374	 Loss: 37.279881	 valid_Loss : 24.240990

Epoch: 375	 Loss: 37.254910	 valid_Loss : 24.202011

Epoch: 376	 Loss: 37.227661	 valid_Loss : 24.323116

Epoch: 377	 Loss: 37.194363	 valid_Loss : 24.506500

Epoch: 378	 Loss: 37.169418	 valid_Loss : 24.657122

Epoch: 379	 Loss: 37.143387	 valid_Loss : 24.782078

Epoch: 380	 Loss: 37.113342	 valid_Loss : 24.951233

Epoch: 381	 Loss: 37.086529	 valid_Loss : 25.195869

Epoch: 382	 Loss: 37.059597	 valid_Loss : 25.486486

Epoch: 383	 Loss: 37.032356	 valid_Loss : 25.656555

Epoch: 384	 Loss: 37.008671	 valid_Loss : 25.561939

Epoch: 385	 Loss: 36.979591	 valid_Loss : 25.368614

Epoch: 386	 Loss: 36.952938	 valid_Loss : 25.274887

Epoch: 387	 Loss: 36.928570	 valid_Loss : 25.322222

Epoch: 388	 Loss: 36.902023	 valid_Loss : 25.451323

Epoch: 389	 Loss: 36.876347	 valid_Loss : 25.585348

Epoch: 390	 Loss: 36.851112	 valid_Loss : 25.6

Epoch: 566	 Loss: 34.679043	 valid_Loss : 23.660904

Epoch: 567	 Loss: 34.674873	 valid_Loss : 23.661762

Epoch: 568	 Loss: 34.670765	 valid_Loss : 23.662380

Epoch: 569	 Loss: 34.666706	 valid_Loss : 23.662451

Epoch: 570	 Loss: 34.662697	 valid_Loss : 23.662901

Epoch: 571	 Loss: 34.658745	 valid_Loss : 23.663960

Epoch: 572	 Loss: 34.654842	 valid_Loss : 23.665403

Epoch: 573	 Loss: 34.650986	 valid_Loss : 23.667004

Epoch: 574	 Loss: 34.647182	 valid_Loss : 23.668762

Epoch: 575	 Loss: 34.643429	 valid_Loss : 23.669968

Epoch: 576	 Loss: 34.639717	 valid_Loss : 23.671551

Epoch: 577	 Loss: 34.636059	 valid_Loss : 23.673578

Epoch: 578	 Loss: 34.632446	 valid_Loss : 23.676456

Epoch: 579	 Loss: 34.628880	 valid_Loss : 23.678972

Epoch: 580	 Loss: 34.625351	 valid_Loss : 23.681284

Epoch: 581	 Loss: 34.621880	 valid_Loss : 23.683142

Epoch: 582	 Loss: 34.618443	 valid_Loss : 23.684944

Epoch: 583	 Loss: 34.615051	 valid_Loss : 23.686895

Epoch: 584	 Loss: 34.611702	 valid_Loss : 23.6

...


Epoch: 1152	 Loss: 34.431137	 valid_Loss : 23.688625

Epoch: 1153	 Loss: 34.431133	 valid_Loss : 23.689512

Epoch: 1154	 Loss: 34.431129	 valid_Loss : 23.690325

Epoch: 1155	 Loss: 34.431126	 valid_Loss : 23.691195

Epoch: 1156	 Loss: 34.431122	 valid_Loss : 23.692314

Epoch: 1157	 Loss: 34.431114	 valid_Loss : 23.693140

Epoch: 1158	 Loss: 34.431110	 valid_Loss : 23.694227

Epoch: 1159	 Loss: 34.431107	 valid_Loss : 23.695389

Epoch: 1160	 Loss: 34.431103	 valid_Loss : 23.696022

Epoch: 1161	 Loss: 34.431095	 valid_Loss : 23.696354

Epoch: 1162	 Loss: 34.431091	 valid_Loss : 23.696339

Epoch: 1163	 Loss: 34.431087	 valid_Loss : 23.696875

Epoch: 1164	 Loss: 34.431080	 valid_Loss : 23.696686

Epoch: 1165	 Loss: 34.431072	 valid_Loss : 23.696339

Epoch: 1166	 Loss: 34.431068	 valid_Loss : 23.695845

Epoch: 1167	 Loss: 34.431068	 valid_Loss : 23.695351

Epoch: 1168	 Loss: 34.431065	 valid_Loss : 23.694408

Epoch: 1169	 Loss: 34.431061	 valid_Loss : 23.694056

Epoch: 1170	 Loss: 34.43105

Epoch: 1345	 Loss: 34.438282	 valid_Loss : 23.659761

Epoch: 1346	 Loss: 34.437443	 valid_Loss : 23.772644

Epoch: 1347	 Loss: 34.433899	 valid_Loss : 23.702051

Epoch: 1348	 Loss: 34.430988	 valid_Loss : 23.713133

Epoch: 1349	 Loss: 34.430374	 valid_Loss : 23.753733

Epoch: 1350	 Loss: 34.431950	 valid_Loss : 23.671631

Epoch: 1351	 Loss: 34.433849	 valid_Loss : 23.770973

Epoch: 1352	 Loss: 34.433674	 valid_Loss : 23.690390

Epoch: 1353	 Loss: 34.432056	 valid_Loss : 23.738695

Epoch: 1354	 Loss: 34.430458	 valid_Loss : 23.743990

Epoch: 1355	 Loss: 34.430569	 valid_Loss : 23.697355

Epoch: 1356	 Loss: 34.431782	 valid_Loss : 23.768772

Epoch: 1357	 Loss: 34.432236	 valid_Loss : 23.701029

Epoch: 1358	 Loss: 34.431511	 valid_Loss : 23.741934

Epoch: 1359	 Loss: 34.430454	 valid_Loss : 23.736366

Epoch: 1360	 Loss: 34.430340	 valid_Loss : 23.702333

Epoch: 1361	 Loss: 34.431004	 valid_Loss : 23.752527

Epoch: 1362	 Loss: 34.431416	 valid_Loss : 23.696795

Epoch: 1363	 Loss: 34.431126

Epoch: 1538	 Loss: 34.430317	 valid_Loss : 23.718603

Epoch: 1539	 Loss: 34.429733	 valid_Loss : 23.695473

Epoch: 1540	 Loss: 34.429268	 valid_Loss : 23.692629

Epoch: 1541	 Loss: 34.429291	 valid_Loss : 23.715775

Epoch: 1542	 Loss: 34.429653	 valid_Loss : 23.677042

Epoch: 1543	 Loss: 34.429916	 valid_Loss : 23.716352

Epoch: 1544	 Loss: 34.429871	 valid_Loss : 23.678839

Epoch: 1545	 Loss: 34.429573	 valid_Loss : 23.700563

Epoch: 1546	 Loss: 34.429276	 valid_Loss : 23.692175

Epoch: 1547	 Loss: 34.429153	 valid_Loss : 23.686411

Epoch: 1548	 Loss: 34.429245	 valid_Loss : 23.709267

Epoch: 1549	 Loss: 34.429428	 valid_Loss : 23.683779

Epoch: 1550	 Loss: 34.429577	 valid_Loss : 23.718166

Epoch: 1551	 Loss: 34.429619	 valid_Loss : 23.689514

Epoch: 1552	 Loss: 34.429520	 valid_Loss : 23.716619

Epoch: 1553	 Loss: 34.429359	 valid_Loss : 23.698029

Epoch: 1554	 Loss: 34.429203	 valid_Loss : 23.708012

Epoch: 1555	 Loss: 34.429108	 valid_Loss : 23.707411

Epoch: 1556	 Loss: 34.429096


Epoch: 1736	 Loss: 34.428848	 valid_Loss : 23.708458

Epoch: 1737	 Loss: 34.428364	 valid_Loss : 23.680275

Epoch: 1738	 Loss: 34.427948	 valid_Loss : 23.695673

Epoch: 1739	 Loss: 34.427704	 valid_Loss : 23.692356

Epoch: 1740	 Loss: 34.427666	 valid_Loss : 23.680529

Epoch: 1741	 Loss: 34.427780	 valid_Loss : 23.700539

Epoch: 1742	 Loss: 34.428005	 valid_Loss : 23.667011

Epoch: 1743	 Loss: 34.428333	 valid_Loss : 23.711967

Epoch: 1744	 Loss: 34.428799	 valid_Loss : 23.664127

Epoch: 1745	 Loss: 34.429474	 valid_Loss : 23.736523

Epoch: 1746	 Loss: 34.430622	 valid_Loss : 23.661367

Epoch: 1747	 Loss: 34.432281	 valid_Loss : 23.776333

Epoch: 1748	 Loss: 34.434258	 valid_Loss : 23.660084

Epoch: 1749	 Loss: 34.436398	 valid_Loss : 23.800808

Epoch: 1750	 Loss: 34.436546	 valid_Loss : 23.674366

Epoch: 1751	 Loss: 34.435020	 valid_Loss : 23.782173

Epoch: 1752	 Loss: 34.431149	 valid_Loss : 23.716022

Epoch: 1753	 Loss: 34.428165	 valid_Loss : 23.715061

Epoch: 1754	 Loss: 34.42780

Epoch: 1933	 Loss: 34.426098	 valid_Loss : 23.795273

Epoch: 1934	 Loss: 34.426041	 valid_Loss : 23.772705

Epoch: 1935	 Loss: 34.426380	 valid_Loss : 23.804678

Epoch: 1936	 Loss: 34.426865	 valid_Loss : 23.774281

Epoch: 1937	 Loss: 34.427418	 valid_Loss : 23.802097

Epoch: 1938	 Loss: 34.427902	 valid_Loss : 23.771450

Epoch: 1939	 Loss: 34.428169	 valid_Loss : 23.813101

Epoch: 1940	 Loss: 34.428104	 valid_Loss : 23.755138

Epoch: 1941	 Loss: 34.427849	 valid_Loss : 23.787933

Epoch: 1942	 Loss: 34.427299	 valid_Loss : 23.750870

Epoch: 1943	 Loss: 34.426785	 valid_Loss : 23.754534

Epoch: 1944	 Loss: 34.426373	 valid_Loss : 23.725080

Epoch: 1945	 Loss: 34.426102	 valid_Loss : 23.739033

Epoch: 1946	 Loss: 34.425938	 valid_Loss : 23.722723

Epoch: 1947	 Loss: 34.425831	 valid_Loss : 23.724117

Epoch: 1948	 Loss: 34.425766	 valid_Loss : 23.729383

Epoch: 1949	 Loss: 34.425732	 valid_Loss : 23.726334

Epoch: 1950	 Loss: 34.425713	 valid_Loss : 23.728210

Epoch: 1951	 Loss: 34.425713

### 모델 2
Attention 적용

In [246]:
class MyModel2(nn.Module):
    def __init__(self):
        super(MyModel2, self).__init__()
        self.fc1 = nn.Linear(29, 16)
        self.btn1 = nn.BatchNorm1d(16)
        self.fc2 = nn.Linear(16, 8)
        self.btn2 = nn.BatchNorm1d(8)
        self.fc3 = nn.Linear(8 ,1)
        
        self.sigmoid = nn.Sigmoid()
        self.at1_1 = nn.Linear(16, 8)
        self.at1_2 = nn.Linear(8, 16)
        self.at2_1 = nn.Linear(8, 4)
        self.at2_2 = nn.Linear(4, 8)
        
        self.act_fn = nn.ELU()
    def forward(self, x):
        x = self.fc1(x)
        x = self.btn1(x)
        x = self.act_fn(x)
        
        attent = self.at1_2(self.at1_1(x))
        atten = self.sigmoid(attent)
        x = attent * x
        
        x = self.fc2(x)
        x = self.btn2(x)
        x = self.act_fn(x)
        
        attent = self.at2_2(self.at2_1(x))
        atten = self.sigmoid(attent)
        x = attent * x
        
        x = self.fc3(x)
        return x
    

model = MyModel2().cuda()
print(model)

MyModel2(
  (fc1): Linear(in_features=29, out_features=16, bias=True)
  (btn1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (btn2): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=8, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (at1_1): Linear(in_features=16, out_features=8, bias=True)
  (at1_2): Linear(in_features=8, out_features=16, bias=True)
  (at2_1): Linear(in_features=8, out_features=4, bias=True)
  (at2_2): Linear(in_features=4, out_features=8, bias=True)
  (act_fn): ELU(alpha=1.0)
)


### 모델 3
어텐션에서 층을 더 쌓음

In [247]:
class MyModel3(nn.Module):
    def __init__(self):
        super(MyModel3, self).__init__()
        self.fc1 = nn.Linear(29, 16)
        self.btn1 = nn.BatchNorm1d(16)
        self.fc2 = nn.Linear(16, 8)
        self.btn2 = nn.BatchNorm1d(8)
        self.fc3 = nn.Linear(8 ,4)
        self.btn3 = nn.BatchNorm1d(4)
        self.fc4 = nn.Linear(4 ,1)
        
        self.sigmoid = nn.Sigmoid()
        self.at1_1 = nn.Linear(16, 8)
        self.at1_2 = nn.Linear(8, 16)
        self.at2_1 = nn.Linear(8, 4)
        self.at2_2 = nn.Linear(4, 8)
        self.at3_1 = nn.Linear(4, 2)
        self.at3_2 = nn.Linear(2, 4)
        
        self.act_fn = nn.ELU()
    def forward(self, x):
        x = self.fc1(x)
        x = self.btn1(x)
        x = self.act_fn(x)
        
        attent = self.at1_2(self.at1_1(x))
        atten = self.sigmoid(attent)
        x = attent * x
        
        x = self.fc2(x)
        x = self.btn2(x)
        x = self.act_fn(x)
        
        attent = self.at2_2(self.at2_1(x))
        atten = self.sigmoid(attent)
        x = attent * x
        
        x = self.fc3(x)
        x = self.btn3(x)
        x = self.act_fn(x)
        
        attent = self.at3_2(self.at3_1(x))
        atten = self.sigmoid(attent)
        x = attent * x
        
        x = self.fc4(x)
        return x
    

model = MyModel3().cuda()
print(model)

MyModel3(
  (fc1): Linear(in_features=29, out_features=16, bias=True)
  (btn1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (btn2): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=8, out_features=4, bias=True)
  (btn3): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=4, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (at1_1): Linear(in_features=16, out_features=8, bias=True)
  (at1_2): Linear(in_features=8, out_features=16, bias=True)
  (at2_1): Linear(in_features=8, out_features=4, bias=True)
  (at2_2): Linear(in_features=4, out_features=8, bias=True)
  (at3_1): Linear(in_features=4, out_features=2, bias=True)
  (at3_2): Linear(in_features=2, out_features=4, bias=True)
  (act_fn): ELU(alpha=1.0)
)


In [248]:
opt = optim.AdamW(model.parameters(), lr=0.001)
Loss_fn = nn.MSELoss()
EPOCHS = 2000

# for epoch in range(1, EPOCHS+1):
#     train(model, train_loader, opt)
#     valid_loss = evaluate(model, valid_loader)
#     print("[EPOCH: {}], \tValidation Loss: {:.4f}\n".format(epoch, valid_loss))
    
for epoch in range(1, EPOCHS+1):
    model.train()
    x = train_x; label = train_y
    x = x.cuda(); label = label.cuda()
    
    opt.zero_grad()
    output = model(x).squeeze()
#     print(output[2].item(),label[2].item())
    loss = torch.sqrt(Loss_fn(output, label))
    loss.backward()
    opt.step()
    
     
    model.eval()
    output = model(valid_x.cuda())
    valid_loss = torch.sqrt(Loss_fn(output, valid_y.cuda()))
    
    print('Epoch: {}\t Loss: {:.6f}\t valid_Loss : {:.6f}\n'.format(epoch, loss, valid_loss))

C:\anaconda3\envs\JSY_GPU\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([4800])) that is different to the input size (torch.Size([4800, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 1	 Loss: 7.841771	 valid_Loss : 47.436226

Epoch: 2	 Loss: 7.814363	 valid_Loss : 6.703139

Epoch: 3	 Loss: 7.794304	 valid_Loss : 5.756507

Epoch: 4	 Loss: 7.763313	 valid_Loss : 6.205737

Epoch: 5	 Loss: 7.736718	 valid_Loss : 6.525544

Epoch: 6	 Loss: 7.728062	 valid_Loss : 6.714735

Epoch: 7	 Loss: 7.718357	 valid_Loss : 6.827067

Epoch: 8	 Loss: 7.718223	 valid_Loss : 6.894207

Epoch: 9	 Loss: 7.703998	 valid_Loss : 6.932062

Epoch: 10	 Loss: 7.694786	 valid_Loss : 6.951356

Epoch: 11	 Loss: 7.687058	 valid_Loss : 6.959404

Epoch: 12	 Loss: 7.678579	 valid_Loss : 6.961239

Epoch: 13	 Loss: 7.671443	 valid_Loss : 6.960198

Epoch: 14	 Loss: 7.666678	 valid_Loss : 6.958207

Epoch: 15	 Loss: 7.661473	 valid_Loss : 6.955750

Epoch: 16	 Loss: 7.652801	 valid_Loss : 6.952878

Epoch: 17	 Loss: 7.641985	 valid_Loss : 6.949580

Epoch: 18	 Loss: 7.632528	 valid_Loss : 6.945849

Epoch: 19	 Loss: 7.625177	 valid_Loss : 6.941741

Epoch: 20	 Loss: 7.618502	 valid_Loss : 6.937569

Epoch: 2


Epoch: 190	 Loss: 6.082333	 valid_Loss : 5.252895

Epoch: 191	 Loss: 6.077284	 valid_Loss : 5.262415

Epoch: 192	 Loss: 6.071136	 valid_Loss : 5.259897

Epoch: 193	 Loss: 6.067325	 valid_Loss : 5.237949

Epoch: 194	 Loss: 6.065989	 valid_Loss : 5.260472

Epoch: 195	 Loss: 6.063612	 valid_Loss : 5.222048

Epoch: 196	 Loss: 6.061071	 valid_Loss : 5.233648

Epoch: 197	 Loss: 6.055097	 valid_Loss : 5.215889

Epoch: 198	 Loss: 6.050752	 valid_Loss : 5.200366

Epoch: 199	 Loss: 6.048676	 valid_Loss : 5.215178

Epoch: 200	 Loss: 6.047556	 valid_Loss : 5.182801

Epoch: 201	 Loss: 6.046922	 valid_Loss : 5.201173

Epoch: 202	 Loss: 6.042929	 valid_Loss : 5.167782

Epoch: 203	 Loss: 6.039697	 valid_Loss : 5.173515

Epoch: 204	 Loss: 6.036007	 valid_Loss : 5.174132

Epoch: 205	 Loss: 6.034182	 valid_Loss : 5.151867

Epoch: 206	 Loss: 6.034131	 valid_Loss : 5.177312

Epoch: 207	 Loss: 6.033051	 valid_Loss : 5.133841

Epoch: 208	 Loss: 6.032061	 valid_Loss : 5.150640

Epoch: 209	 Loss: 6.027821	 va


Epoch: 372	 Loss: 5.927191	 valid_Loss : 4.964517

Epoch: 373	 Loss: 5.926325	 valid_Loss : 4.964608

Epoch: 374	 Loss: 5.925488	 valid_Loss : 4.961026

Epoch: 375	 Loss: 5.924609	 valid_Loss : 4.955571

Epoch: 376	 Loss: 5.923816	 valid_Loss : 4.957580

Epoch: 377	 Loss: 5.922967	 valid_Loss : 4.958400

Epoch: 378	 Loss: 5.922093	 valid_Loss : 4.961044

Epoch: 379	 Loss: 5.921259	 valid_Loss : 4.958060

Epoch: 380	 Loss: 5.920369	 valid_Loss : 4.954778

Epoch: 381	 Loss: 5.919524	 valid_Loss : 4.951938

Epoch: 382	 Loss: 5.918689	 valid_Loss : 4.954779

Epoch: 383	 Loss: 5.917838	 valid_Loss : 4.955005

Epoch: 384	 Loss: 5.917035	 valid_Loss : 4.956732

Epoch: 385	 Loss: 5.916229	 valid_Loss : 4.950392

Epoch: 386	 Loss: 5.915580	 valid_Loss : 4.962999

Epoch: 387	 Loss: 5.915876	 valid_Loss : 4.946847

Epoch: 388	 Loss: 5.918499	 valid_Loss : 4.998096

Epoch: 389	 Loss: 5.927009	 valid_Loss : 4.949000

Epoch: 390	 Loss: 5.924910	 valid_Loss : 4.933760

Epoch: 391	 Loss: 5.917147	 va

Epoch: 556	 Loss: 5.870572	 valid_Loss : 4.876193

Epoch: 557	 Loss: 5.870560	 valid_Loss : 4.875653

Epoch: 558	 Loss: 5.870553	 valid_Loss : 4.875431

Epoch: 559	 Loss: 5.870540	 valid_Loss : 4.877028

Epoch: 560	 Loss: 5.870523	 valid_Loss : 4.877787

Epoch: 561	 Loss: 5.870512	 valid_Loss : 4.876719

Epoch: 562	 Loss: 5.870504	 valid_Loss : 4.877847

Epoch: 563	 Loss: 5.870493	 valid_Loss : 4.877774

Epoch: 564	 Loss: 5.870479	 valid_Loss : 4.880874

Epoch: 565	 Loss: 5.870466	 valid_Loss : 4.884811

Epoch: 566	 Loss: 5.870457	 valid_Loss : 4.887073

Epoch: 567	 Loss: 5.870448	 valid_Loss : 4.889376

Epoch: 568	 Loss: 5.870439	 valid_Loss : 4.890226

Epoch: 569	 Loss: 5.870428	 valid_Loss : 4.893247

Epoch: 570	 Loss: 5.870415	 valid_Loss : 4.894625

Epoch: 571	 Loss: 5.870404	 valid_Loss : 4.895952

Epoch: 572	 Loss: 5.870394	 valid_Loss : 4.894556

Epoch: 573	 Loss: 5.870384	 valid_Loss : 4.891078

Epoch: 574	 Loss: 5.870375	 valid_Loss : 4.889464

Epoch: 575	 Loss: 5.870367	 val

...

Epoch: 1231	 Loss: 5.867337	 valid_Loss : 4.879254

Epoch: 1232	 Loss: 5.867335	 valid_Loss : 4.879429

Epoch: 1233	 Loss: 5.867332	 valid_Loss : 4.880008

Epoch: 1234	 Loss: 5.867330	 valid_Loss : 4.880606

Epoch: 1235	 Loss: 5.867328	 valid_Loss : 4.881724

Epoch: 1236	 Loss: 5.867326	 valid_Loss : 4.882488

Epoch: 1237	 Loss: 5.867323	 valid_Loss : 4.884441

Epoch: 1238	 Loss: 5.867321	 valid_Loss : 4.886807

Epoch: 1239	 Loss: 5.867319	 valid_Loss : 4.888458

Epoch: 1240	 Loss: 5.867316	 valid_Loss : 4.889338

Epoch: 1241	 Loss: 5.867314	 valid_Loss : 4.889594

Epoch: 1242	 Loss: 5.867311	 valid_Loss : 4.889372

Epoch: 1243	 Loss: 5.867309	 valid_Loss : 4.889363

Epoch: 1244	 Loss: 5.867307	 valid_Loss : 4.889268

Epoch: 1245	 Loss: 5.867304	 valid_Loss : 4.888204

Epoch: 1246	 Loss: 5.867302	 valid_Loss : 4.887097

Epoch: 1247	 Loss: 5.867300	 valid_Loss : 4.886801

Epoch: 1248	 Loss: 5.867297	 valid_Loss : 4.886716

Epoch: 1249	 Loss: 5.867295	 valid_Loss : 4.885631

Epoch: 1250	


Epoch: 1394	 Loss: 5.867390	 valid_Loss : 4.863401

Epoch: 1395	 Loss: 5.867173	 valid_Loss : 4.863591

Epoch: 1396	 Loss: 5.867337	 valid_Loss : 4.865280

Epoch: 1397	 Loss: 5.867172	 valid_Loss : 4.866922

Epoch: 1398	 Loss: 5.867302	 valid_Loss : 4.866556

Epoch: 1399	 Loss: 5.867172	 valid_Loss : 4.866213

Epoch: 1400	 Loss: 5.867262	 valid_Loss : 4.866435

Epoch: 1401	 Loss: 5.867170	 valid_Loss : 4.867271

Epoch: 1402	 Loss: 5.867234	 valid_Loss : 4.868188

Epoch: 1403	 Loss: 5.867176	 valid_Loss : 4.869012

Epoch: 1404	 Loss: 5.867211	 valid_Loss : 4.870327

Epoch: 1405	 Loss: 5.867176	 valid_Loss : 4.871336

Epoch: 1406	 Loss: 5.867190	 valid_Loss : 4.872076

Epoch: 1407	 Loss: 5.867175	 valid_Loss : 4.873325

Epoch: 1408	 Loss: 5.867170	 valid_Loss : 4.873917

Epoch: 1409	 Loss: 5.867170	 valid_Loss : 4.873927

Epoch: 1410	 Loss: 5.867153	 valid_Loss : 4.874344

Epoch: 1411	 Loss: 5.867165	 valid_Loss : 4.873922

Epoch: 1412	 Loss: 5.867141	 valid_Loss : 4.873820

Epoch: 1413


Epoch: 1556	 Loss: 5.866804	 valid_Loss : 4.862858

Epoch: 1557	 Loss: 5.866804	 valid_Loss : 4.863021

Epoch: 1558	 Loss: 5.866807	 valid_Loss : 4.863512

Epoch: 1559	 Loss: 5.866826	 valid_Loss : 4.862541

Epoch: 1560	 Loss: 5.866890	 valid_Loss : 4.864332

Epoch: 1561	 Loss: 5.867100	 valid_Loss : 4.863372

Epoch: 1562	 Loss: 5.867853	 valid_Loss : 4.872979

Epoch: 1563	 Loss: 5.869814	 valid_Loss : 4.873792

Epoch: 1564	 Loss: 5.876196	 valid_Loss : 4.886678

Epoch: 1565	 Loss: 5.872794	 valid_Loss : 4.878964

Epoch: 1566	 Loss: 5.872085	 valid_Loss : 4.890748

Epoch: 1567	 Loss: 5.867486	 valid_Loss : 4.901518

Epoch: 1568	 Loss: 5.867537	 valid_Loss : 4.902435

Epoch: 1569	 Loss: 5.869685	 valid_Loss : 4.917056

Epoch: 1570	 Loss: 5.867217	 valid_Loss : 4.927409

Epoch: 1571	 Loss: 5.867826	 valid_Loss : 4.925174

Epoch: 1572	 Loss: 5.868836	 valid_Loss : 4.931088

Epoch: 1573	 Loss: 5.866958	 valid_Loss : 4.942604

Epoch: 1574	 Loss: 5.868713	 valid_Loss : 4.930245

Epoch: 1575

Epoch: 1719	 Loss: 5.868170	 valid_Loss : 4.882124

Epoch: 1720	 Loss: 5.867436	 valid_Loss : 4.886821

Epoch: 1721	 Loss: 5.866680	 valid_Loss : 4.894897

Epoch: 1722	 Loss: 5.867631	 valid_Loss : 4.887232

Epoch: 1723	 Loss: 5.867727	 valid_Loss : 4.889256

Epoch: 1724	 Loss: 5.866684	 valid_Loss : 4.894221

Epoch: 1725	 Loss: 5.867530	 valid_Loss : 4.883533

Epoch: 1726	 Loss: 5.867676	 valid_Loss : 4.886507

Epoch: 1727	 Loss: 5.866690	 valid_Loss : 4.894688

Epoch: 1728	 Loss: 5.867609	 valid_Loss : 4.883662

Epoch: 1729	 Loss: 5.867525	 valid_Loss : 4.887846

Epoch: 1730	 Loss: 5.866728	 valid_Loss : 4.896839

Epoch: 1731	 Loss: 5.867687	 valid_Loss : 4.882686

Epoch: 1732	 Loss: 5.867290	 valid_Loss : 4.882573

Epoch: 1733	 Loss: 5.866797	 valid_Loss : 4.890443

Epoch: 1734	 Loss: 5.867631	 valid_Loss : 4.878172

Epoch: 1735	 Loss: 5.867018	 valid_Loss : 4.876248

Epoch: 1736	 Loss: 5.866875	 valid_Loss : 4.883530

Epoch: 1737	 Loss: 5.867408	 valid_Loss : 4.874173

Epoch: 1738	


Epoch: 1877	 Loss: 5.866436	 valid_Loss : 4.897354

Epoch: 1878	 Loss: 5.866586	 valid_Loss : 4.892251

Epoch: 1879	 Loss: 5.866551	 valid_Loss : 4.891998

Epoch: 1880	 Loss: 5.866426	 valid_Loss : 4.892138

Epoch: 1881	 Loss: 5.866547	 valid_Loss : 4.886981

Epoch: 1882	 Loss: 5.866530	 valid_Loss : 4.885502

Epoch: 1883	 Loss: 5.866416	 valid_Loss : 4.884707

Epoch: 1884	 Loss: 5.866508	 valid_Loss : 4.879649

Epoch: 1885	 Loss: 5.866513	 valid_Loss : 4.878836

Epoch: 1886	 Loss: 5.866408	 valid_Loss : 4.877260

Epoch: 1887	 Loss: 5.866467	 valid_Loss : 4.874170

Epoch: 1888	 Loss: 5.866496	 valid_Loss : 4.874256

Epoch: 1889	 Loss: 5.866405	 valid_Loss : 4.873306

Epoch: 1890	 Loss: 5.866426	 valid_Loss : 4.871391

Epoch: 1891	 Loss: 5.866472	 valid_Loss : 4.871260

Epoch: 1892	 Loss: 5.866407	 valid_Loss : 4.870852

Epoch: 1893	 Loss: 5.866392	 valid_Loss : 4.870341

Epoch: 1894	 Loss: 5.866437	 valid_Loss : 4.870917

Epoch: 1895	 Loss: 5.866411	 valid_Loss : 4.870586

Epoch: 1896

# Test 데이터 입력

In [249]:
model.eval()
output = model(torch.tensor(test_x).cuda()).squeeze()

output

tensor([4.3584, 4.3327, 4.0701,  ..., 4.4797, 4.1123, 4.2821], device='cuda:0',
       grad_fn=<SqueezeBackward0>)

In [250]:
submission = pd.read_csv('data/sample_submission.csv')

output = output.cpu().detach().numpy()

submission['INVC_CONT'] = output

submission

,index,INVC_CONT
0,32000,4.358434
1,32001,4.332676
2,32002,4.070079
3,32003,4.497986
4,32004,4.277174
...,...,...
4635,36635,4.856220
4636,36636,4.237500
4637,36637,4.479704
4638,36638,4.112335


In [251]:
submission.to_csv('JSY_OUTPUT2.csv',index = False)